# Speech to Text Testing Notebook

Dependencies

Pip (env)
- google-cloud-speech[libcst]
- ffmpeg-python


Conda 
- nb_conda_kernels

Conda (env)
- Python 3.10
- ipykernel


## Imports

In [5]:
from google.cloud import speech
import ffmpeg
import sys
# from tqdm.notebook import tqdm

## Video Conversion

Using MP4 (lossy codec) can reduce the accuracy of the audio. In order to generate the best results, we will transcode the video into lossless codec (FLAC). I decided not to go with LINEAR16 since its use case lies in applications that need more higher quality audio (also results in higher file size)

In [6]:
import os
base_dir = os.getcwd()
print(base_dir)

C:\Users\ricky\Documents\My Files\Altum Labs\autolab\sst_transcription


In [16]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import ipywidgets as widgets
import numpy as np

In [23]:
probe = ffmpeg.probe('in.mp4')

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [14]:
# input_file = r'sst_transcription/sec4.mp4'
# output_file = r'output.flac'
{
    ffmpeg
    .input('sec4.mp4')
    .hflip()
    .output('output.mp4')
    .run()
}
# stream = ffmpeg.input(input_file)
# stream = ffmpeg.hflip(stream)
# stream = ffmpeg.output(stream, 'output.mp4')
# ffmpeg.run(stream)

FileNotFoundError: [WinError 2] The system cannot find the file specified

## Speech2Text 

In [ ]:
client = speech.SpeechClient()

request = speech.RecognizeRequest(request={"config": config, "audio": audio})
response = client.list_voices(request=request)